In [ ]:
import numpy as np
import scipy.stats as stats
from hansolo2 import *

# np.random.seed(10)

#Coder le learning blocs !



Nb = 100
expert = "PWA"  # "EWA" or "PWA"
order="epoch" # "epoch": objects presented by random blocs, or "random": each object is randomly chosen with replacement
learning = "sequential" # "normal": weights of every layer are updated after each object, or "blocs": weights of every layer are 
# updated after each bloc of objects, or "sequential": weights of hidden layer are updated during M1 obj and then weights of output layer are updated during M1 objects

M = 1000

#for sequential, M1 M2 chosen such that M1+M2=M
M1=500
M2=M-M1

N = 1000
p = 0.4  # spiking proba input neurons 
nu = 0 # instantaneous spiking proba


n_input = 4 #nb input neurons
# green red circle square
K_input = 2*n_input # nb connections to a mid neuron 
K_mid = 4 # nb mid neurons
# (green, circle) (green, square) (red, circle) (red, square)
K_output = 2  # nb output neurons

sup =  4 * p
#sup = p
eta_output = np.sqrt(8 * np.log(K_mid) / M) / sup  # para EWA

sup_mid = 2 * p**2
eta_mid = np.sqrt(8 * np.log(K_input) / M) / sup_mid

obj = [
    [1, 0, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 1, 0, 1],
]
# green circle green square red circle red square


answers = np.zeros((Nb, M))  # 0 if wrong, 1 if correct, -1 if sequential and output didn't start

W_output = np.zeros((K_output, K_mid, M, Nb))  # weights output neurons
W_output[:, :, 0, :] += 1 / K_mid  # initialisation
W_mid = np.zeros((K_mid, K_input, M, Nb))
W_mid[:, :, 0, :] += 1 / K_input  # initialisation

P_output = np.zeros((K_output, M, Nb))  # empirical spiking proba output neurons, stay at 0 for sequential at the beginning
P_mid = np.zeros((K_mid, M, Nb))

# para_PWA=2*np.log(K_input) para opti PWA for regret
para_PWA = 2

## SV: use list comprehension

for nb in range(Nb):
   
    if order == 'epoch':
        L=list_obj_random(obj,M)
    else:
        L=list_objects_random(obj,M)


    W_output_not_renorm = np.zeros((K_output, K_mid, M)) 
    W_output_not_renorm[:, :, 0] += 1


    W_mid_not_renorm = np.zeros((K_mid, K_input, M))
    W_mid_not_renorm[:, :, 0] += 1

    P_input = np.zeros((n_input, M))  # empirical spiking proba input neurons

    cum_gain_output = np.zeros(K_output)
    cum_gain_mid = np.zeros((K_mid))
    cum_gain_mid_to_output = np.zeros((K_output, K_mid))
    cum_gain_input_to_mid = np.zeros((K_mid,K_input))

    for m in range(M):
        #print(W_output_not_renorm[:,:,m])
        #print(W_output[:,:,m,nb])
        #print(W_mid_not_renorm[:,:,m])
        #print(W_mid[:,:,m,nb])

        obj_cur = L[m]# current object

        # simulation of the input neurons
        input_neurons = np.zeros((n_input, N))
        cur_act = np.where(obj_cur == 1)[0]
        input_neurons[cur_act, :] = Poisson(p, (cur_act.size, N))

        # simulation of the mid neurons
        mid_neurons = np.zeros((K_mid,N))
        for i in range(K_mid):
            mid_neurons[i,:] = Hawkes(input_neurons, W_mid[i,:,m,nb], nu)

        # empirical spiking proba
        P_input[:, m] = np.sum(input_neurons, axis=1) / N
        P_mid[:, m, nb] = np.sum(mid_neurons, axis=1) / N

        if learning != "sequential" or m < M1:
            gain_mid = Gain_Mid(K_mid,K_input, input_neurons)
            cum_gain_input_to_mid += gain_mid
            cum_gain_mid += np.sum(W_mid[:,:,m,nb] * gain_mid, axis = 1)


            # simulation of the output neurons
            ## SV: note -> harder to vectorize :( Doable?
        
        if learning != "sequential" or m >= M1:
            output_neurons = np.zeros((K_output, N))
            for i in range(K_output):
                output_neurons[i, :] = Hawkes_lin(mid_neurons, W_output[i, :, m, nb])
            P_output[:, m, nb] = np.sum(output_neurons, axis=1) / N

            if (in_A(obj_cur) and P_output[0, m, nb] > P_output[1, m, nb]) or (in_B(obj_cur) and P_output[1, m, nb] > P_output[0, m, nb]):
                answers[nb, m] = 1

            gain_output = Gain_Output(K_output, K_mid, P_mid[:, m, nb], obj_cur)
            cum_gain_mid_to_output += gain_output
            cum_gain_output += np.sum(W_output[:, :, m, nb] * gain_output, axis=1)

        

        if m < M - 1:
            if expert == "EWA":
                if learning != "sequential" or m < M1:
                    W_mid_not_renorm[:, :, m + 1] = EWA(W_mid_not_renorm[:, :, m], eta_mid, gain_mid, K_mid)
                else :
                    W_mid_not_renorm[:, :, m + 1] = W_mid_not_renorm[:, :, m]

                if learning != "sequential" or m >= M1:
                    W_output_not_renorm[:, :, m + 1] = EWA(W_output_not_renorm[:, :, m], eta_output, gain_output, K_output)
                else :
                    W_output_not_renorm[:, :, m + 1] = W_output_not_renorm[:, :, m] 

            elif expert == "PWA":
                if learning != "sequential" or m < M1:
                    W_mid_not_renorm[:, :, m + 1] = PWA(para_PWA, K_mid, K_input, cum_gain_mid, cum_gain_input_to_mid)
                else :
                    W_mid_not_renorm[:, :, m + 1] = W_mid_not_renorm[:, :, m]

                if learning != "sequential" or m >= M1:    
                    W_output_not_renorm[:, :, m + 1] = PWA(para_PWA, K_output, K_mid, cum_gain_output, cum_gain_mid_to_output)
                else :
                    W_output_not_renorm[:, :, m + 1] = W_output_not_renorm[:, :, m] 

            for j in range(K_output):
                if np.sum(W_output_not_renorm[j, :, m + 1]) == 0 :
                    W_output[j, :, m + 1, nb] = 1/K_mid
                else :
                    W_output[j, :, m + 1, nb] = W_output_not_renorm[j, :, m + 1] / np.sum(W_output_not_renorm[j, :, m + 1])
                    
            for j in range(K_mid):
                if np.sum(W_mid_not_renorm[j, :, m + 1]) == 0 :
                    W_mid[j, :, m + 1, nb] = 1/K_input
                else:
                    W_mid[j, :, m + 1, nb] = W_mid_not_renorm[j, :, m + 1] / np.sum(W_mid_not_renorm[j, :, m + 1])

        
    print(nb)

np.save(f"/Users/sophiejaffard/Desktop/saves2/{expert}_{order}_{learning}", answers)
np.save(f"/Users/sophiejaffard/Desktop/saves2/{expert}_{order}__{learning}W_output", W_output[:,:,:,Nb-1])
np.save(f"/Users/sophiejaffard/Desktop/saves2/{expert}_{order}__{learning}W_mid", W_mid[:,:,:,Nb-1])
np.save(f"/Users/sophiejaffard/Desktop/saves2/{expert}_{order}__{learning}P_output", P_output[:,:,Nb-1])
np.save(f"/Users/sophiejaffard/Desktop/saves2/{expert}_{order}__{learning}P_mid", P_mid[:,:,Nb-1])
np.save(f"/Users/sophiejaffard/Desktop/saves2/{expert}_{order}__{learning}L", L)



